In [46]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm import tqdm
from VX_merge_treasury_rates import *
import ctypes

%load_ext autoreload
%autoreload 2


path1 = "../_data/issue_data.parquet"
path2 = "../_data/monthly_ts_data.parquet"

wrds_path = "../../FS-project-files/wrds_bond.parquet"
red_code_path = "../../FS-project-files/RED_and_ISIN_mapping.parquet"

issue_df = pd.read_parquet(path1)
treas_df = pd.read_parquet(path2)
bond_df = pd.read_parquet(wrds_path)
red_df = pd.read_parquet(red_code_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
# test 1

t_df = generate_treasury_data(issue_df, treas_df)

In [37]:
t_df.head()

,kycrspid,kytreasno,mcaldt,tmpubout,tmatdt,treas_yld
0,19610622.800000,200001.0,1960-10-31,NaN,1961-06-22,0.028055
1,19610622.800000,200001.0,1960-11-30,NaN,1961-06-22,0.029241
2,19610622.800000,200001.0,1960-12-30,NaN,1961-06-22,0.024777
3,19610622.800000,200001.0,1961-01-31,NaN,1961-06-22,0.025169
4,19610622.800000,200001.0,1961-02-28,NaN,1961-06-22,0.027243


In [38]:
bond_df.columns

Index(['offering_date', 'company_symbol', 'maturity', 'amount_outstanding',
       'security_level', 'yield', 'cusip', 'isin', 'conv', 'offering_price',
       'price_eom', 't_spread', 'maturity_time_frame', 'rating'],
      dtype='object')

In [39]:
# test 2

ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)

bond_treas_df = merge_treasuries_into_bonds(bond_df, t_df)

ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)



100%|██████████| 1285559/1285559 [08:52<00:00, 2414.38it/s]


-2147483646

In [ ]:
# test 3

fin_df = merge_red_code_into_bond_treas(bond_treas_df, red_df)

In [51]:
pickle_path = "../../FS-project-files/merged_bond_treas_red.pkl"
fin_df.to_pickle(pickle_path)

In [45]:
red_df.columns

Index(['redcode', 'ticker', 'obl_cusip', 'isin', 'tier'], dtype='object')

In [6]:
red_df.columns

Index(['redcode', 'ticker', 'obl_cusip', 'isin', 'tier'], dtype='object')

In [5]:
bond_df.head()

,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,offering_price,price_eom,t_spread,maturity_time_frame,rating
date,,,,,,,,,,,,,,
2002-07-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.048266,000361AB1,US000361AB18,0.0,100.0,102.791,NaN,10,1
2002-08-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.043858,000361AB1,US000361AB18,0.0,100.0,103.089,NaN,10,1
2002-09-30,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.041222,000361AB1,US000361AB18,0.0,100.0,103.143,NaN,10,1
2002-10-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.038728,000361AB1,US000361AB18,0.0,100.0,103.143,NaN,10,1
2002-11-30,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.040151,000361AB1,US000361AB18,0.0,100.0,102.750,0.034101,10,1


# What I need to do

Build a function that merges the treasury report dates and maturities. 

## Report dates:
normalize by year and month, all of them are end of month but just off by a little bit.

## Maturity dates:
Look for dates within 3 days of the maturity date. Hopefully this does not drop a significant amount.


In [20]:
bond_df['amount_outstanding'].describe()

count    1.370635e+06
mean     5.410310e+05
std      6.076376e+05
min      1.000000e+04
25%      1.500000e+05
50%      4.000000e+05
75%      7.500000e+05
max      1.100000e+07
Name: amount_outstanding, dtype: float64

In [21]:
bond_df.reset_index().dropna(subset=['yield'])

,date,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,offering_price,price_eom,t_spread,maturity_time_frame,rating
0,2002-07-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.048266,000361AB1,US000361AB18,0.0,100.0,102.791000,NaN,10,1
1,2002-08-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.043858,000361AB1,US000361AB18,0.0,100.0,103.089000,NaN,10,1
2,2002-09-30,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.041222,000361AB1,US000361AB18,0.0,100.0,103.143000,NaN,10,1
3,2002-10-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.038728,000361AB1,US000361AB18,0.0,100.0,103.143000,NaN,10,1
4,2002-11-30,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.040151,000361AB1,US000361AB18,0.0,100.0,102.750000,0.034101,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370630,2024-03-31,2024-03-25,BCS,2026-03-27,13748.0,SEN,0.003740,06745Q3E7,US06745Q3E77,0.0,100.0,99.250000,NaN,2,1
1370631,2024-03-31,2024-03-25,BCS,2029-03-29,11765.0,SEN,0.003428,06748L453,US06748L4538,0.0,100.0,9.828117,NaN,5,1
1370632,2024-04-30,2024-03-25,BCS,2029-03-29,11765.0,SEN,0.003485,06748L453,US06748L4538,0.0,100.0,9.828117,NaN,5,1
1370633,2024-05-31,2024-03-25,BCS,2029-03-29,11765.0,SEN,0.003547,06748L453,US06748L4538,0.0,100.0,9.828117,NaN,5,1


In [22]:
bond_df.columns

Index(['offering_date', 'company_symbol', 'maturity', 'amount_outstanding',
       'security_level', 'yield', 'cusip', 'isin', 'conv', 'offering_price',
       'price_eom', 't_spread', 'maturity_time_frame', 'rating'],
      dtype='object')

In [23]:
t_mod = treas_df.merge(issue_df, on=['kytreasno', 'kycrspid'], how='left')

In [24]:
t_mod = t_mod.dropna(subset=['mcaldt', 'tmyld', 'tmatdt', 'kycrspid', 'kytreasno'])

In [25]:
date_set = set(t_mod['mcaldt'])
mat_set = set(t_mod['tmatdt'])

In [26]:
t_mod

,kycrspid,kytreasno,mcaldt,tmpubout,tmyld,tmatdt
0,19610622.800000,200001.0,1960-10-31,NaN,0.000076,1961-06-22
1,19610622.800000,200001.0,1960-11-30,NaN,0.000079,1961-06-22
2,19610622.800000,200001.0,1960-12-30,NaN,0.000067,1961-06-22
3,19610622.800000,200001.0,1961-01-31,NaN,0.000068,1961-06-22
4,19610622.800000,200001.0,1961-02-28,NaN,0.000074,1961-06-22
...,...,...,...,...,...,...
192056,20260122.400000,208343.0,2025-01-31,NaN,0.000112,2026-01-22
192057,20270131.204120,208344.0,2025-01-31,NaN,0.000114,2027-01-31
192058,20280115.204250,208345.0,2025-01-31,NaN,0.000115,2028-01-15
192059,20300131.204250,208346.0,2025-01-31,NaN,0.000117,2030-01-31


In [27]:
# filters necessary for bond_df
mask_am_out = bond_df['amount_outstanding'] >= 100_000 # filtering for accurate pricing, no need for max
# mask_price_eom1 = bond_df['price_eom'] >= 50 # filtering for ratio vs face value
# mask_price_eom2 = bond_df['price_eom'] <= 500 # filtering for unreasonable higher limit for price
mask_yield1 = bond_df['yield'] >= 0 # ensure reasonable yield, above 0
mask_yield2 = bond_df['yield'] <= 0.5 # massive discounting is unreasonable


# need to filter yield for Nans and large values




In [28]:
bond_test_df = bond_df[mask_am_out & mask_yield1 & mask_yield2]

bond_test_df.shape

(1056171, 14)

In [29]:
bond_test_df.head()

,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,offering_price,price_eom,t_spread,maturity_time_frame,rating
date,,,,,,,,,,,,,,
2002-07-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.085600,001957AM1,US001957AM13,0.0,99.265,97.213129,0.014847,10,1
2002-08-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.062781,001957AM1,US001957AM13,0.0,99.265,100.684813,0.011224,10,1
2002-09-30,1994-03-23,T,2004-04-01,400000.0,SEN,0.066960,001957AM1,US001957AM13,0.0,99.265,100.066504,0.007308,10,1
2002-10-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.066670,001957AM1,US001957AM13,0.0,99.265,100.112053,0.007682,10,0
2002-11-30,1994-03-23,T,2004-04-01,123856.0,SEN,0.055671,001957AM1,US001957AM13,0.0,99.265,101.500000,0.008068,10,0


In [30]:
len(bond_test_df['maturity'].unique())

4265

In [31]:
mat_set = {pd.to_datetime(d).date() for d in mat_set}

# Get unique maturity dates
maturity_dates = pd.Series(bond_test_df['maturity'].unique())

# Check if any maturity date falls within ±3 days of any mat_date in mat_set
valid_maturities = maturity_dates.apply(
    lambda x: any(mat_date - timedelta(days=3) <= x <= mat_date + timedelta(days=3) for mat_date in mat_set)
)


In [32]:
valid_maturities.sum()

3535

In [33]:
v_mat_set = maturity_dates[valid_maturities].tolist()  # Convert valid dates to list

# Use .isin() to filter bond_test_df
bond_no_df = bond_test_df[bond_test_df['maturity'].isin(v_mat_set)]

# Display the filtered DataFrame
bond_no_df

,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,offering_price,price_eom,t_spread,maturity_time_frame,rating
date,,,,,,,,,,,,,,
2002-07-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.085600,001957AM1,US001957AM13,0.0,99.265,97.213129,0.014847,10,1
2002-08-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.062781,001957AM1,US001957AM13,0.0,99.265,100.684813,0.011224,10,1
2002-09-30,1994-03-23,T,2004-04-01,400000.0,SEN,0.066960,001957AM1,US001957AM13,0.0,99.265,100.066504,0.007308,10,1
2002-10-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.066670,001957AM1,US001957AM13,0.0,99.265,100.112053,0.007682,10,0
2002-11-30,1994-03-23,T,2004-04-01,123856.0,SEN,0.055671,001957AM1,US001957AM13,0.0,99.265,101.500000,0.008068,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,2024-02-29,BAC,2025-04-25,788121.0,SEN,0.010543,09710N440,US09710N4401,0.0,100.000,9.913272,NaN,1,1
2024-07-31,2024-02-29,BAC,2025-04-25,788121.0,SEN,0.011770,09710N440,US09710N4401,0.0,100.000,9.913272,NaN,1,1
2024-03-31,2024-03-08,CGMHI,2025-06-11,117590.0,SEN,0.005178,17291LTZ8,US17291LTZ84,0.0,100.000,99.375000,NaN,1,1


In [34]:
treas_test_df = t_mod.copy()
bond_t_df = bond_test_df.copy()

In [35]:
bond_t_df = bond_test_df.reset_index()
bond_t_df['year_m'] = bond_t_df['date'].dt.to_period('M')

treas_test_df['mcaldt'] = pd.to_datetime(treas_test_df['mcaldt'])
treas_test_df['year_m'] = treas_test_df['mcaldt'].dt.to_period('M')

In [36]:
len(treas_test_df['kytreasno'].unique())

8056

In [37]:
bond_t_df.head()

,date,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,offering_price,price_eom,t_spread,maturity_time_frame,rating,year_m
0,2002-07-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.085600,001957AM1,US001957AM13,0.0,99.265,97.213129,0.014847,10,1,2002-07
1,2002-08-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.062781,001957AM1,US001957AM13,0.0,99.265,100.684813,0.011224,10,1,2002-08
2,2002-09-30,1994-03-23,T,2004-04-01,400000.0,SEN,0.066960,001957AM1,US001957AM13,0.0,99.265,100.066504,0.007308,10,1,2002-09
3,2002-10-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.066670,001957AM1,US001957AM13,0.0,99.265,100.112053,0.007682,10,0,2002-10
4,2002-11-30,1994-03-23,T,2004-04-01,123856.0,SEN,0.055671,001957AM1,US001957AM13,0.0,99.265,101.500000,0.008068,10,0,2002-11


In [41]:
# Step 1: Reset index and extract year-month for efficient merging
bond_t_df = bond_test_df.reset_index()
bond_t_df['year_m'] = bond_t_df['date'].dt.to_period('M')

treas_test_df['mcaldt'] = pd.to_datetime(treas_test_df['mcaldt'])
treas_test_df['year_m'] = treas_test_df['mcaldt'].dt.to_period('M')
treas_test_df['v_id'] = treas_test_df.index

# Step 2: Pre-filter treasuries to only include relevant months
treas_test_df = treas_test_df[treas_test_df['year_m'].isin(bond_t_df['year_m'].unique())]

bond_t_df = bond_t_df.sort_values('date')
treas_test_df = treas_test_df.sort_values('mcaldt')

treas_dict = {y_m: treas_test_df[treas_test_df.year_m == y_m] for y_m in bond_t_df['year_m'].unique()}

lookup_cache = {}

def get_v_id(year_m, maturity):
    """ Function to find the best-matching v_id for each unique (year_m, maturity) pair """
    if (year_m, maturity) in lookup_cache:
        return lookup_cache[(year_m, maturity)]

    t_df = treas_dict.get(year_m, pd.DataFrame()).copy()

    if t_df.empty or pd.isna(maturity):
        lookup_cache[(year_m, maturity)] = np.NaN
        return np.NaN

    t_df['tmatdt'] = pd.to_datetime(t_df['tmatdt'], errors='coerce')
    maturity_date = pd.to_datetime(maturity, errors='coerce')

    if t_df['tmatdt'].isna().all() or pd.isna(maturity_date):
        lookup_cache[(year_m, maturity)] = np.NaN
        return np.NaN

    t_df['day_diff'] = abs((t_df['tmatdt'] - maturity_date).dt.days)
    t_df = t_df[t_df['day_diff'] <= 3]

    if t_df.empty:
        lookup_cache[(year_m, maturity)] = np.NaN
        return np.NaN

    if t_df['tmpubout'].isna().all():
        lookup_cache[(year_m, maturity)] = t_df.iloc[0]['v_id']
        return t_df.iloc[0]['v_id']

    max_row_idx = t_df['tmpubout'].idxmax()
    lookup_cache[(year_m, maturity)] = t_df.loc[max_row_idx, 'v_id']
    return t_df.loc[max_row_idx, 'v_id']

tqdm.pandas()
bond_t_df['v_id'] = bond_t_df.progress_apply(lambda row: get_v_id(row.year_m, row.maturity), axis=1)

merge_df = bond_t_df.merge(treas_test_df, on='v_id', how='left')

merge_df.head()


100%|██████████| 1056171/1056171 [06:30<00:00, 2706.18it/s]


,date,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,...,rating,year_m_x,v_id,kycrspid,kytreasno,mcaldt,tmpubout,tmyld,tmatdt,year_m_y
0,2002-07-31,1994-03-23,T,2004-04-01,400000.0,SEN,0.085600,001957AM1,US001957AM13,0.0,...,1,2002-07,74424.0,20040331.203620,203475.0,2002-07-31,25000.0,0.000055,2004-03-31,2002-07
1,2002-07-31,2002-05-15,KFT,2007-06-01,1000000.0,SEN,0.042161,50075NAG9,US50075NAG97,0.0,...,1,2002-07,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
2,2002-07-31,2000-11-08,EAS,2010-11-15,200000.0,SEN,0.062695,29266MAA7,US29266MAA71,0.0,...,1,2002-07,88156.0,20101115.512750,203977.0,2002-07-31,2821.0,0.000084,2010-11-15,2002-07
3,2002-07-31,2002-05-15,KFT,2012-06-01,1500000.0,SEN,0.055111,50075NAH7,US50075NAH70,0.0,...,1,2002-07,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
4,2002-07-31,2002-05-15,HI,2012-05-15,1750000.0,SEN,0.074660,441812JY1,US441812JY13,0.0,...,1,2002-07,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT


In [43]:
merge_df = merge_df.dropna(subset=['v_id'])

In [44]:
merge_path = "/Users/xuv14/Winter 2025 FINM/Full Stack/FS-project-files/bond_merge_treas.parquet"

merge_df.to_parquet(merge_path)